<a href="https://colab.research.google.com/github/devaine/CodeRED2024/blob/main/LLaMa_2_Prompting_Guide_with_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

In this Colab Notebook, we are going to chat with Llama-2 7B.

By the end of this tutorial, you'll be able to interact with this model and use it to generate conversational responses.

Whether you're curious about chatbot technology or simply want to see a machine-generated response to a particular question, this notebook will serve as a comprehensive guide.

## Workflow
1. **Installations**: We'll begin by setting up our environment with the required libraries.
2. **Prerequisites**: Ensure we have access to the Llama-2 7B model on Hugging Face.
3. **Loading the Model & Tokenizer**: Retrieve the model and tokenizer for our session.
4. **Creating the Llama Pipeline**: Prepare our model for generating responses.
5. **Interacting with Llama through Gradio's ChatInterface**: Prompt the model for answers and explore its capabilities.

Let's dive in!

**First, change runtime to GPU.**


You can play with Llama-2 7B Chat here: https://huggingface.co/spaces/huggingface-projects/llama-2-7b-chat

## Installations

Before we proceed, we need to ensure that the essential libraries are installed:
- `Hugging Face Transformers`: Provides us with a straightforward way to use pre-trained models.
- `PyTorch`: Serves as the backbone for deep learning operations.
- `Accelerate`: Optimizes PyTorch operations, especially on GPU.

In [1]:
!pip install transformers torch accelerate

   ---------------------------------------- 0.0/279.7 kB ? eta -:--:--
   ---------------------------------------  276.5/279.7 kB 8.6 MB/s eta 0:00:01
   ---------------------------------------- 279.7/279.7 kB 5.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


To use `gr.ChatInterface()` we need the newest Gradio

In [2]:
!pip install --upgrade gradio

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/50.6 kB ? eta -:--:--
     ---------------------------------------- 50.6/50.6 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/83.5 kB ? eta -:--:--
     ---------------------------------------- 83.5/83.5 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/45.9 kB ? eta -:--:--
     ---------------------------------------- 45.9/45.9 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/124.7 kB ? eta -:--:--
     -------------------------------------- 124.7/124.7 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.8 MB ? eta -:--:--
    --------------------------------------- 0.4/16.8 MB 8.3 MB/s et

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
asyncpraw 7.7.1 requires aiofiles<1, but you have aiofiles 23.2.1 which is incompatible.
cyberdrop-dl 5.1.45 requires aiofiles==0.8.0, but you have aiofiles 23.2.1 which is incompatible.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.20.3 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


If `!pip install --upgrade gradio` returns an error that says: `NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968`, do the following:
1. Uncomment the next cell.
2. Run the cell
3. Restart Runtime: `Runtime -> Restart Runtime`

In [ ]:
# import locale
# locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

### Prerequisites

To load our desired model, `meta-llama/Llama-2-7b-chat-hf`, we first need to authenticate ourselves on Hugging Face. This ensures we have the correct permissions to fetch the model.

1. Gain access to the model on Hugging Face: [Link](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).
2. Use the Hugging Face CLI to login and verify your authentication status.



In [2]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!huggingface-cli whoami

Not logged in


### Loading Model & Tokenizer

Here, we are preparing our session by loading both the Llama model and its associated tokenizer.

The tokenizer will help in converting our text prompts into a format that the model can understand and process.

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-chat-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:631: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


### Creating the Llama Pipeline

We'll set up a pipeline for text generation.

This pipeline simplifies the process of feeding prompts to our model and receiving generated text as output.

*Note*: This cell takes 2-3 minutes to run

In [ ]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Comparing Methods: Basic vs. Advanced

Before diving into our advanced method for conversational interaction, let's explore a basic method for generating responses using the `get_response()` function. We will later discuss its limitations and how the advanced method overcomes them.





In [ ]:
def get_response(prompt: str) -> None:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
    )
    print("Chatbot:", sequences[0]['generated_text'])



In [ ]:
get_response("Hi, I'm Kris")

Chatbot: Hi, I'm Kris. Here are some of the best ways to get your ex back:
1. Give them space: If your ex has broken up with you, it's important to give them the space they need. Respect their boundaries and don't try to contact them for a while. This will give them time to process their feelings and think about what they want.
2. Show that you've changed: If you've made mistakes in the past, it's important to show your ex that you've changed and grown as a person. This can involve working on yourself, improving your behavior, and being more mindful of their needs.
3. Be patient: Getting your ex back can take time, so it's important to be patient and not rush things. Give them the time and space they need to come around, and don't try to force them into anything.
4. Be kind and respectful: Treat your ex with kindness and respect, even if they've hurt you in the past. Avoid being confrontational or aggressive, and try to maintain a positive attitude.
5. Communicate openly and honestly: 

In [ ]:
get_response("What's my name?")

Chatbot: What's my name?

Answer: Your name is Jack.


### Drawbacks of `get_response()`

1. **Lack of Conversation History**: The basic approach does not account for past interactions, making it less effective for maintaining a coherent conversation.
2. **Limited Customization**: The function doesn't allow for advanced prompt formatting or handling system-level instructions.
3. **Not Ready for UI Integration**: This basic approach isn't designed for easy integration with user interface libraries like Gradio.

## Improved Prompts

The right structure of Llama 2 prompts:

```
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_message }} [/INST]
```

### Building the Prompt

Explaining the parameters:
- `message` is the current message we're sending
- `history` is the history of conversation as a list of tupples `[(user_msg1, bot_msg1), (usr_msg2, bot_msg2), ...]`


In [ ]:
SYSTEM_PROMPT = """<s>[INST] <<SYS>>
You are a helpful bot. Your answers are clear and concise.
<</SYS>>

"""

# Formatting function for message and history
def format_message(message: str, history: list, memory_limit: int = 3) -> str:
    """
    Formats the message and history for the Llama model.

    Parameters:
        message (str): Current message to send.
        history (list): Past conversation history.
        memory_limit (int): Limit on how many past interactions to consider.

    Returns:
        str: Formatted message string
    """
    # always keep len(history) <= memory_limit
    if len(history) > memory_limit:
        history = history[-memory_limit:]

    if len(history) == 0:
        return SYSTEM_PROMPT + f"{message} [/INST]"

    formatted_message = SYSTEM_PROMPT + f"{history[0][0]} [/INST] {history[0][1]} </s>"

    # Handle conversation history
    for user_msg, model_answer in history[1:]:
        formatted_message += f"<s>[INST] {user_msg} [/INST] {model_answer} </s>"

    # Handle the current message
    formatted_message += f"<s>[INST] {message} [/INST]"

    return formatted_message

### Getting Responses

We need the function to generate responses.

In [ ]:
# Generate a response from the Llama model
def get_llama_response(message: str, history: list) -> str:
    """
    Generates a conversational response from the Llama model.

    Parameters:
        message (str): User's input message.
        history (list): Past conversation history.

    Returns:
        str: Generated response from the Llama model.
    """
    query = format_message(message, history)
    response = ""

    sequences = llama_pipeline(
        query,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
    )

    generated_text = sequences[0]['generated_text']
    response = generated_text[len(query):]  # Remove the prompt from the output

    print("Chatbot:", response.strip())
    return response.strip()


In [ ]:
import gradio as gr

gr.ChatInterface(get_llama_response).launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

### Conclusion

Thanks to the Hugging Face Library, creating a pipeline to chat with llama 2 (or any other open-source LLM) is quite easy.

But if you worked a lot with much larger models such as GPT-4, you need to adjust your expectations.